In [3]:
import ast
from asdl.asdl import ASDLGrammar
from asdl.lang.py.py_asdl_helper import *
from asdl.lang.py.py_transition_system import *
from asdl.hypothesis import *
import astor

In [4]:
# read in the grammar specification of Python 3, defined in ASDL
asdl_text = open('./asdl/lang/py3/py3_asdl.txt').read()
grammar = ASDLGrammar.from_text(asdl_text)

In [5]:
py_code = """pandas.read('file.csv', nrows=100)"""
py_ast = ast.parse(py_code)

### AST2Actions

In [6]:
# convert to adsl ast, different from normal ast
asdl_ast = python_ast_to_asdl_ast(py_ast.body[0], grammar)

In [7]:
# get python transition parser
parser = PythonTransitionSystem(grammar)

# get all actions from ast
actions = parser.get_actions(asdl_ast)

In [8]:
actions

[ApplyRule[stmt -> Expr(expr value)],
 ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)],
 ApplyRule[expr -> Attribute(expr value, identifier attr)],
 ApplyRule[expr -> Name(identifier id, expr_context ctx)],
 GenToken[pandas],
 ApplyRule[expr_context -> Load()],
 GenToken[read],
 ApplyRule[expr -> Str(string s)],
 GenToken[file.csv],
 GenToken[</primitive>],
 Reduce,
 ApplyRule[keyword -> keyword(identifier? arg, expr value)],
 GenToken[nrows],
 ApplyRule[expr -> Num(object n)],
 GenToken[100],
 Reduce]

In [11]:
actions[0].__class__

asdl.transition_system.ApplyRuleAction

In [18]:
hypothesis = Hypothesis()
parser.get_valid_continuation_types(hypothesis)

(asdl.transition_system.ApplyRuleAction,)

### Actions2AST

In [23]:
# a hypothesis is an (partial) ASDL AST generated using a sequence of tree-construction actions
hypothesis = Hypothesis()
for t, action in enumerate(actions, 1):

    # this assert the next action is a valid type
    assert action.__class__ in parser.get_valid_continuation_types(hypothesis)

    # this assert that next action has a valid field
    if isinstance(action, ApplyRuleAction) and hypothesis.frontier_node:
        assert action.production in grammar[hypothesis.frontier_field.type]
        # grammar[hypothesis.frontier_field.type] is a list of possible actions

    # print('t=%d, Action=%s' % (t, action))
    hypothesis.apply_action(action)

In [24]:
ast_from_actions = asdl_ast_to_python_ast(hypothesis.tree, grammar)

In [25]:
astor.to_source(ast_from_actions).strip()

"pandas.read('file.csv', nrows=100)"